In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep 14 23:41:26 2016

@author: WANG Shaoyang
"""

import csv
import numpy as np


def read():
    path = 'C:/Users/SyW/Desktop/BiShe/'
    with open(path + 'DAX-price-2-years.csv', 'rb') as price_csv, open(path + 'DAX-sentiment-2-years.csv', 'rb') as sentiment_csv:
        price_reader = csv.reader(price_csv)
        sentiment_reader = csv.reader(sentiment_csv)

        trends = {}
        last_price = 0
        for row in price_reader:
            price = float(row[4])
            date = row[0]
            if last_price != 0:
                trends[date] = 1 if price > last_price else 0
            last_price = price
        m = len(trends.keys())  # total data

        sentiments = {}
        event_id = {}

        event_count = {}

        n = 0  # total characteristics
        for row in sentiment_reader:
            date = row[0]
            event = (int(row[1]), int(row[2]), int(row[3]))
            if event not in event_id:
                event_id[event] = n
                n += 1

                event_count[event] = 1
            else:
                event_count[event] += 1

            weight = int(row[4])
            if date not in sentiments:
                sentiments[date] = {}
            sentiments[date][event] = weight

    i = 0
    for event in event_count.keys():
        if event_count[event] < 200:
            event_id[event] = -1
            n -= 1
        else:
            event_id[event] = i
            i += 1

    return trends, sentiments, event_id, m, n


def build_vectors(trends, sentiments, event_id, m, n):
    X_all = np.zeros((m, n + 1))
    y_all = np.zeros(m)

    dates = sorted(trends.keys())
    for i in xrange(0, m):
        date = dates[i]
        for event in sentiments[date]:
            if event_id[event] != -1:
                X_all[i, event_id[event]] = sentiments[date][event]
            # X_all[i, event_id[event]] = sentiments[date][event]
        X_all[i, n] = 1
        y_all[i] = trends[date]

    return X_all, y_all


def h(theta, x):
    return 1 / (1 + np.exp(-np.dot(x, theta) / theta.shape[0] * 0.0001))  # x can be a vector or a matrix


def train(X, y):
    m = X.shape[0]
    n = X.shape[1]
    theta = np.zeros(n)
    alpha = 0.1

    for j in xrange(50000):
        #theta_ori = np.copy(theta)
        for i in xrange(m):
            theta += alpha * (y[i] - h(theta, X[i])) * X[i]
        #print np.sum(np.abs(theta_ori - theta))

    print '[' + ','.join([str(x) for x in theta]) + ']'
    return theta


def test(X, y, theta):
    total = y.shape[0]
    correct = 0
    for i in xrange(total):
        yh = h(theta, X[i])
        yh = 1 if yh > 0.5 else 0
        if yh == y[i]:
            correct += 1
    print 'Total: %d, Correct: %d, Accuracy: %f' % (total, correct, float(correct) / total)


if __name__ == '__main__':
    training_num = 300
    validation_num = training_num + 100

    # read trends and sentiments from file
    trends, sentiments, event_id, m, n = read()

    # build vectors
    X_all, y_all = build_vectors(trends, sentiments, event_id, m, n)

    # work out theta
    X = X_all[0:training_num]
    y = y_all[0:training_num]
    theta = train(X, y)

    # validation
    X_test = X_all[training_num:validation_num]
    y_test = y_all[training_num:validation_num]
    test(X_test, y_test, theta)

    # test
    X_test = X_all[validation_num:]
    y_test = y_all[validation_num:]
    test(X_test, y_test, theta)


[356.277539688,-74.6407171905,-6660.93814353,139.345094058,-781.441414924,8631.82756647,-3988.61270727,2819.80634468,1626.75261626,155.330864697,1082.29647987,8650.36280509,-5539.07393147,1966.30785299,5752.30938797,-388.461796771,-658.04225582,4239.17396896,3150.30851649,599.953354038,-4193.83800332,792.062246807,556.823002066,-2922.57254463,-1197.80477592,2411.63434293,5802.10167426,-122.013985314,-45.0276623205,-930.306042114,3914.57002126,-1463.59514959,390.219281273,-872.098367356,6840.00088772,77.0084968869,-3280.45882341,105.781335943,729.630638082,5608.5342237,-3179.68243443,1680.41799186,-1446.66776655,1720.08282698,819.148285867,-639.305730151,302.157051097,5537.25740049,2442.60445135,-1767.52890327,-3289.06403564,500.501181783,7450.15878392,-1394.99841348,432.175336524,9469.70207808,839.100214806,2973.06233703,-174.402996964,-1500.8102103,1578.09703273,-6050.59533242,-1654.14345434,119.281558751,-11770.9875841,-15575.5845382,1949.53103257,521.561127908,-2160.46937283,1648.47